In [1]:
# importing the modules
from datetime import date, timedelta, datetime
from pymongo import MongoClient
import san
from bson.objectid import ObjectId

client = MongoClient('mongodb://localhost:27017/')
db1 = client.geeksforgeeks
data = {}

with open("IndexCoins.idx") as openfileobject:
    for line in openfileobject:
        
        # One by one File's cryptocurrency names are read
        id = line.strip()
        try:
            print(id)
            
            # Collecting the data for 7 days and hence range(7) is taken
            for idx in range(7):
                daystr = str(date.today() - timedelta(days = idx))

                # Coll_santiment_Price collection documents need to be created
                # It will have columns namely "_id", "time", "id",
                # "priceBtc", "priceUsd", "volume", "marketcap"
                data['id'] = id
                data['time'] = daystr

                # _id column for unique key and can be generated by using ObjectId()
                data['_id'] = ObjectId()
                
                # Initially create 3 columns in collection as rest of the
                # columns are updated after running santiment api call			
                db1.Coll_santiment_Price.insert(data)
            try:
                # Santiment API call to get Cryptocurrency prices
                daa = san.get("prices/" + id,
                            from_date = "2020-08-20",
                            to_date = "2020-08-27",
                            interval = "1d")
                
                # API call output and for bitcoin it is given in
                # https://media.geeksforgeeks.org/wp-content/uploads/20200827191739/bitcoinprice.png
                print(daa)
                
            except:
                print("URL error")
                continue;

            # 7 days output
            for idx in range(7):
                
                daystr = str(date.today() - timedelta(days=idx))

                # API call output for the above chosen date
                row = daa.loc[daystr]
                
                # priceBtc, priceUsd, volume and marketcap are
                # collected and stored in seperate variables
                priceBtc = row['priceBtc']
                priceUsd = row['priceUsd']
                volume = row['volume']
                marketcap = row['marketcap']
                print(id, daystr, priceBtc, priceUsd, volume, marketcap)
                try:
                    # Update the collection with above details against
                    # cryptocurrency id and time i.e. bitcoin and 2020-08-27
                    db1.Coll_santiment_Price.update(
                        {'time': daystr, 'id': id},
                        {"$set": {"priceBtc": priceBtc,
                                "priceUsd": priceUsd,
                                "volume": volume,
                                "marketcap": marketcap,
                                }
                        },
                        upsert = True
                        )
                except Exception as e:
                    print(e)
        except:
            print("Error")


ModuleNotFoundError: No module named 'pymongo'